# Starbucks Data Analyst Project

I am posing as a Starbucks big data analyst looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.


In [1]:

# pip install git+https://github.com/Kaggle/learntools.git


In [2]:
import math
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim            
# from learntools.geospatial.tools import Nominatim 

import folium 
from folium import Marker
from folium.plugins import MarkerCluster


Using the `embed_map()` function to visualize maps.

In [3]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

In [4]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../Starbucks_Locations/input/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [5]:
# Run check to identify rows in each column with missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


Next we will fill in these values with the Nominatim geocoder.

We are using `Nominatim()` (from `geopy.geocoders`) to geocode values.
https://geocoder.readthedocs.io/providers/OpenStreetMap.html

In [6]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)


### 2) View Berkeley locations.

Let's take a look at the locations we just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [7]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Add a marker for each Berkeley location
for index, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker(location=[row['Latitude'], row['Longitude']],popup= row['Store Name']).add_to(m_2)


# Show the map
m_2

### 3) Consolidate Geo data.

Next we load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [8]:
CA_counties = gpd.read_file("../starbucks_locations/input/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [9]:
CA_pop = pd.read_csv("input/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("input/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("input/CA_county_median_age.csv", index_col="GEOID")

We will join the `CA_counties` GeoDataFrame with the `CA_pop`, `CA_high_earners`, and `CA_median_age` dataframes.

We rename the new GeoDataFrame `CA_stats`, and make sure it has 8 columns. 
Using the `ESPG 4326` for coordinates in latitude & longitude

In [10]:
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")
CA_stats.crs = {'init': 'epsg:4326'}

C:\ProgramData\Anaconda3\envs\map_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  We will create a "density" column which represents the population density.

In [11]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
6,6097,Sonoma County,4578.952090,"POLYGON ((-122.93507 38.31396, -122.93511 38.3...",499942,32713,41.4,109.182623
7,6031,Kings County,3604.052342,"POLYGON ((-119.95894 36.25547, -119.95894 36.2...",151366,2943,31.5,41.998835
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
9,6061,Placer County,3890.821444,"POLYGON ((-121.06545 39.00654, -121.06538 39.0...",393149,28334,41.6,101.045243


### 4) Which counties look promising?

Collapsing all of the information into a single GeoDataFrame also makes it much easier to select counties that meet specific criteria.

We will create a GeoDataFrame `sel_counties` that contains a subset of the rows (and all of the columns) from the `CA_stats` GeoDataFrame.  In particular,we would select counties where:
- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

In [12]:
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]


### 5) How many stores did we identify?

When looking for the next Starbucks Reserve Roastery location, we would consider all of the pre-existing stores within the selected counties.  So, how many stores are within the selected counties?

In [13]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))

#  EPSG 4326 corresponds to coordinates in latitude and longitude.
starbucks_gdf.crs = {'init': 'epsg:4326'}

C:\ProgramData\Anaconda3\envs\map_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


So, how many stores are in the selected counties based on our selection criteria?

In [14]:
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)

num_stores

1043

### 6) Visualize the store locations.

Creating a map that shows the locations of the stores that we identified in the previous step.

In [15]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Show the map
m_6

This notebook is modified after the `Geospatial Analysis` course from Kaggle 

https://www.kaggle.com/code/alexisbcook/manipulating-geospatial-data/tutorial

The data sets used are obtaained from Kaggle as well

https://www.kaggle.com/learn/geospatial-analysis

Link to my repository hosting the project

https://github.com/EpsilonZulu/Starbucks_Locations